In [4]:
pip install lxml

     |████████████████████████████████| 5.4 MB 2.5 MB/s eta 0:00:01     |█████████▏                      | 1.6 MB 2.5 MB/s eta 0:00:02
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd

In [4]:
sample_df = pd.read_html("./bargage.html")[1]

In [11]:
DAY_NIGHT_COLUMN = 3
sample_df.iloc[:, DAY_NIGHT_COLUMN] = sample_df.iloc[:, DAY_NIGHT_COLUMN].map(lambda x: x=="(夜)").astype(int)

In [14]:
weekdic = {"月":0, "火":1, "水":2, "木":3, "金":4, "土":5, "日":6}

In [60]:
sample_df.head()

町 名          可燃       不燃 資源  ペット 有価物        番地詳細
  町 名  町 名.1   可燃 可燃.1  不燃 資源  ペット 有価物        番地詳細
0   あ  旭町1丁目  0,3    1  2木       水   水         NaN
1   あ  旭町2丁目  0,3    1  4木       水   水  1～18,23～27
2   あ  旭町2丁目  0,3    1  4木       水   木       19～22
3   あ  旭町3丁目  0,3    1  4木       水   水        1～21
4   あ  旭町3丁目  0,3    1  4木       水   木       22の一部

In [58]:
# sample_df.iloc[:, BURNABLE_WD]

In [35]:
# 可燃
BURNABLE_WD = 2

In [59]:
sample_df.iloc[:, BURNABLE_WD] = sample_df.iloc[:, BURNABLE_WD].map(
            lambda x: ",".join([str(weekdic[wd]) for wd in list(x)])
)

In [61]:
# 不燃
NON_BURNABLE_WD = 4

non_burn_df = sample_df.iloc[:, NON_BURNABLE_WD].str.split("", expand=True).iloc[:, 1:3]

In [62]:
non_burn_df.columns = ("nth_week", "weekday_info")

In [63]:
sample_df["nth_week"] = non_burn_df.nth_week
sample_df["weekday_info"] = non_burn_df.weekday_info.map(weekdic)

In [64]:
RESOURCE = 5

sample_df.iloc[:, RESOURCE] = sample_df.iloc[:, RESOURCE].map(weekdic)
sample_df.iloc[:, RESOURCE].unique()

array([2, 0, 4, 3])

In [65]:
pd.concat([sample_df.iloc[:, RESOURCE], sample_df.iloc[:, RESOURCE].map(weekdic)], axis=1).drop_duplicates()

資源  ペット        
   資源  ペット 資源  ペット
0        2     NaN
8        0     NaN
14       4     NaN
19       3     NaN

In [66]:
VALUABLES = 6

sample_df.iloc[:, VALUABLES] = sample_df.iloc[:, VALUABLES].map(weekdic)
sample_df.iloc[:, VALUABLES].unique()

array([2, 3, 0, 1])

In [68]:
DISTRICT_COL = 7
sample_df.iloc[:, DISTRICT_COL]

0              NaN
1       1～18,23～27
2            19～22
3             1～21
4            22の一部
5              1～6
6             7～22
7              NaN
8              NaN
9       673のみ米ケ崎町会
10             NaN
11             NaN
12             NaN
13             NaN
14             NaN
15      1～11,23～39
16           12～22
17           京葉線沿い
18     400番台 春日神社側
19             NaN
20    1,8～11,30～33
21       2～5,14～28
22             NaN
23             NaN
24             NaN
25             NaN
26             NaN
Name: (番地詳細, 番地詳細), dtype: object

In [89]:
def get_json(row):
    out_list = []
    for garbage_type in range(1, 4):
        nth_week = -1 # 不燃ごみのみ利用する。他のごみは毎週収集する
        day_or_night = -1 # 昼夜の指定がされているのは可燃ごみだけ
        area_id = row[1]
        if garbage_type == 1:
            day_or_night = row.iloc[DAY_NIGHT_COLUMN]
            weekday_info = row.iloc[BURNABLE_WD]
            
        elif garbage_type == 2: # non_burnable
            nth_week = row.iloc[8]
            weekday_info = row.iloc[9]
            
        elif garbage_type == 3: # resources / 資源
            weekday_info = row.iloc[RESOURCE]
            
        elif garbage_type == 4: # 有価物
            weekday_info = row.iloc[VALUABLES]

        out_list.append({
            "area_id": area_id,
            "garbage_type": garbage_type,
            "day_or_night":day_or_night,
            "nth_week":nth_week,
            "weekday_info":weekday_info,
        })
    return out_list


In [90]:
sample_df.apply(get_json, axis=1)[0]

[{'area_id': '旭町1丁目',
  'garbage_type': 1,
  'day_or_night': 1,
  'nth_week': -1,
  'weekday_info': '0,3'},
 {'area_id': '旭町1丁目',
  'garbage_type': 2,
  'day_or_night': -1,
  'nth_week': '2',
  'weekday_info': 3},
 {'area_id': '旭町1丁目',
  'garbage_type': 3,
  'day_or_night': -1,
  'nth_week': -1,
  'weekday_info': 2}]

In [88]:
sample_df.columns.shape, sample_df.columns

((10,),
 MultiIndex([(         '町 名',     '町 名'),
             (         '町 名',   '町 名.1'),
             (          '可燃',      '可燃'),
             (          '可燃',    '可燃.1'),
             (          '不燃',      '不燃'),
             (     '資源  ペット', '資源  ペット'),
             (         '有価物',     '有価物'),
             (        '番地詳細',    '番地詳細'),
             (    'nth_week',        ''),
             ('weekday_info',        '')],
            ))